# In the standard terminology used when talking about A*,

**g(n) represents the exact cost of the path from the starting point to any vertex n, 
and
**h(n) represents the heuristic estimated cost from vertex n to the goal.


best first search stores the nodes in fringe priority queue , 
order is an important thing "f(n) = g(n) + h(n)"

f(n) is responsible for the order ... then we select the optimal node from fringe (priority queue)

In [2]:
"""
    An implementation of A* Search Algorithm to solve the 8-Puzzle Problem
"""
from queue import PriorityQueue
import time

In [3]:
class Board(object):
    """
    - This class defines a board for the 8-Puzzle.
    - Tiles are denoted using 1-8, 0 denotes a blank tile.
    """
    #done
    def __init__(self, board=None, moves=0, previous=None):
        """
            board: array representing the current board,
            moves: number of moves to get to this board,
            previous: previous state of the board
        """
        if board is None:
            self.board = [1, 2, 3, 4, 5, 6, 7, 8, 0]
        else:
            self.board = board
            
        self.previous = previous
        self.moves = moves
        
        
    #done
    def is_goal(self):
        """
            returns True if current board is goal state
        """
        for i in range(0, 9):
            if i != 8:
                if self.board[i] != i + 1:
                    return False

        return True

    #done
    def move_blank(self, where):
        """
            where: Move blank 'left', 'right',
                    'up', or 'down',
                   Does nothing if the move is out-of-bounds.
                   
            calling exchange function to swap two tiles
        """
        
        blank = self.find_blank()

        if where == 'left':
            if blank % 3 != 0:
                t_col = (blank % 3) - 1
                t_row = int(blank / 3)
                self.exchange(blank, t_row * 3 + t_col)

        if where == 'right':
            if blank % 3 != 2:
                t_col = (blank % 3) + 1
                t_row = int(blank / 3)
                self.exchange(blank, t_row * 3 + t_col)

        if where == 'up':
            if int(blank / 3) != 0:
                t_col = (blank % 3)
                t_row = int(blank / 3) - 1
                self.exchange(blank, t_row * 3 + t_col)

        if where == 'down':
            if int(blank / 3) != 2:
                t_col = (blank % 3)
                t_row = int(blank / 3) + 1
                self.exchange(blank, t_row * 3 + t_col)

    #done
    def find_blank(self):
        """
            returns index of blank tile
        """
        blank = None
        for i in range(0, 9):
            if self.board[i] == 0:
                blank = i
                break
        return blank

    #done
    def clone(self):
        """
            returns copy of the current board with
                moves = current moves + 1 and
                previous = current board
        """
        return Board(self.board.copy(), self.moves + 1, self)

    #done
    def exchange(self, source, target):
        """
            exchange the 'source' tile with 'target'
        """
        # print('Exchanging: {} <-> {}'.format(source, target))
        #swap two variables in python 
        self.board[source], self.board[target] = self.board[target], self.board[source]

    #done
    def neighbours(self):
        """
            returns a list of all valid neighbours generated by moving
                the blank tile once in all possible directions
        """
        blank_index = self.find_blank()

        neighbours = []

        # print('Blank found: {}, := {}, {}'.format(blank_index, int(blank_index / 3), blank_index % 3))
        # Can we move blank tile left?
        if blank_index % 3 != 0:
            new_board = self.clone()
            new_board.move_blank('left')
            neighbours.append(new_board)

        # right?
        if blank_index % 3 != 2:
            new_board = self.clone()
            new_board.move_blank('right')
            neighbours.append(new_board)

        # up?
        if int(blank_index / 3) != 0:
            new_board = self.clone()
            new_board.move_blank('up')
            neighbours.append(new_board)

        # down?
        if int(blank_index / 3) != 2:
            new_board = self.clone()
            new_board.move_blank('down')
            neighbours.append(new_board)

        return neighbours

    #done
    def manhattan(self):
        """
            returns manhattan distance for the board
        """
        manhattan = 0
        for i in range(0, 9):
            if self.board[i] != i + 1 and self.board[i] != 0:
                correct_pos = 8 if self.board[i] == 0 else self.board[i] - 1
                s_row = int(i / 3)
                s_col = i % 3
                t_row = int(correct_pos / 3)
                t_col = correct_pos % 3
                manhattan += abs(s_row - t_row) + abs(s_col - t_col)

        return manhattan
    
    #done
    def misplaced(self):
        """
            returns the number of misplaced tiles
        """
        misplaced = 0
        for i in range(0, 9):
            if self.board[i] != i + 1 or self.board[i] == 0:
                misplaced += 1
                
        return misplaced
    
    
    

    def to_pq_entry(self, count,mode="manhattan"):
        """
            returns the tuple (priority, count, board)
            
            this functions helps store neighbours ordered by heuristic 
        """
        if mode == "manhattan":
            return (self.moves + self.manhattan(), count, self)
        elif mode == "misplaced":
            return (self.moves + self.misplaced(), count, self)

    def __str__(self):
        """
            Same as print(self) except this returns a string
        """
        string = ''
        string = string + '+---+---+---+\n'
        for i in range(3):
            for j in range(3):
                tile = self.board[i * 3 + j]
                string = string + '| {} '.format(' ' if tile == 0 else tile)
            string = string + '|\n'
            string = string + '+---+---+---+\n'
        return string

    def __eq__(self, other):
        """
            check if self == other
        """
        if other is None:
            return False
        else:
            return self.board == other.board

    def get_previous_states(self):
        """
            return a list of previous states by going up the state space tree
        """
        states = [self]
        prev = self.previous
        while prev is not None:
            states.append(prev)
            prev = prev.previous

        states.reverse()
        return states



In [4]:
def AStarTreeSolve(initial_board,mode):
    """
        returns a list of moves from 'initial_board' to goal state
            calculated using A* algorithm
    """
    queue = PriorityQueue()
    queue.put(initial_board.to_pq_entry(0,mode))

    i = 1
    while not queue.empty(): 
        board = queue.get()[2] #get returns (priority, count, board) [2]retrieve the board 
        
        if not board.is_goal():
            for neighbour in board.neighbours():
                #print("mode : " + mode)
                queue.put(neighbour.to_pq_entry(i,mode))
                #to_pq_entry returns (priority,count,neighbourBoard) 
                #put the tuple into the priority queue so now we have the lowest cost first in the queue  
                i += 1
        else:
            return board.get_previous_states()

    return None

def AStarGraphSolve(initial_board,mode):
    """
        returns a list of moves from 'initial_board' to goal state
            calculated using A* algorithm
    """
    queue = PriorityQueue()
    queue.put(initial_board.to_pq_entry(0,mode))

    AllNodes = []
    
    i = 1
    while not queue.empty(): 
        board = queue.get()[2] #get returns (priority, count, board) [2]retrieve the board 
        
        if not board.is_goal():
            for neighbour in board.neighbours():
                #this condition prevents revisiting the succesor
                if neighbour != board.previous:
                    #print("mode : " + mode)
                    queue.put(neighbour.to_pq_entry(i,mode))
                    #to_pq_entry returns (priority,count,neighbourBoard) 
                    #put the tuple into the priority queue so now we have the lowest cost first in the queue  
                    i += 1
        else:
            return board.get_previous_states()

    return None




In [35]:
def main():
    initial = Board([ 1, 2, 3, 5, 6, 0, 7, 8, 4])
    
    #print(str(initial))
    
    #print(initial.is_goal())
    
    #mode "misplaced" or "manhattan"
    
    #tree
    Start_time = int(round(time.time() * 1000))
    solved = AStarTreeSolve(initial,"misplaced") #change misplaced or manhattan
    End_time = int(round(time.time() * 1000))
    TreeTime = End_time - Start_time
    
    print("***** TreePathSearch *****")
    print("*****" , TreeTime ,"milisec *****")
    for s in solved :
        print(str(s))
        
    #Graph
    Start_time = int(round(time.time() * 1000))
    solved = AStarGraphSolve(initial,"misplaced") #change misplaced or manhattan
    End_time = int(round(time.time() * 1000))
    GraphTime = End_time - Start_time
    
    
    print("\n\n\n***** GraphPathSearch *****")
    print("*****" , GraphTime ,"milisec *****")
    for s in solved :
        print(str(s))
        
        
        
    print("***** TreePathSearch *****")
    print("*****" , TreeTime ,"milisec*****")
    print("***** GraphPathSearch *****")
    print("*****" , GraphTime ,"milisec *****")

In [36]:
main()

***** TreePathSearch *****
***** 2472 milisec *****
+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 5 | 6 |   |
+---+---+---+
| 7 | 8 | 4 |
+---+---+---+

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 5 |   | 6 |
+---+---+---+
| 7 | 8 | 4 |
+---+---+---+

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
|   | 5 | 6 |
+---+---+---+
| 7 | 8 | 4 |
+---+---+---+

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 7 | 5 | 6 |
+---+---+---+
|   | 8 | 4 |
+---+---+---+

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 7 | 5 | 6 |
+---+---+---+
| 8 |   | 4 |
+---+---+---+

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 7 | 5 | 6 |
+---+---+---+
| 8 | 4 |   |
+---+---+---+

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 7 | 5 |   |
+---+---+---+
| 8 | 4 | 6 |
+---+---+---+

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 7 |   | 5 |
+---+---+---+
| 8 | 4 | 6 |
+---+---+---+

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 7 | 4 | 5 |
+---+---+---+
| 8 |   | 6 |
+---+---+---+

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 7 | 4 | 5 |
+